In [1]:
import string
import os
import pandas as pd
import datetime
import numpy as np
import statsmodels.api as sm
import math
from scipy.linalg import svd
#import Functions as Fun
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
###Some parameters
mu=0.2;
tol=1e-10;


In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
####for training
nums= 30
start_dates = [pd.to_datetime('1957-03-01')+pd.offsets.DateOffset(years=x) for x in range(nums) ]

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
####f_gradh
def f_gradh(w,X,y,mu):
    r = np.matmul(X,w)-y['RET']
    ind0 = np.where(abs(r)<= mu)
    ind0 = pd.Series(np.asarray(ind0)[0])
    ind1 = np.where(r>mu)
    ind1 = pd.Series(np.asarray(ind1)[0])

    indf1 = np.where(r< (-mu))
    indf1 = pd.Series(np.array(indf1)[0])
    grad = np.matmul(np.transpose(X.iloc[ind0,:]),(np.matmul(X.iloc[ind0,:],w)-y.iloc[ind0,0]))+np.matmul(np.transpose(mu*X.iloc[ind1,:]),np.ones(len(ind1)))-np.matmul(np.transpose(mu*X.iloc[indf1,:]),np.ones(len(indf1)))
    return grad

In [ ]:
### proximal_l   only for lasso

def proximalH_l(groups,nc,xtest,mtrain,ytest,w,X,y,mu,tol,L,l2,func):
    w= np.asarray(w[0])
    dim = X.shape[0]
    max_iter = 3000
    gamma= 1/L
    l1 = l2

    v= w
    yhatbig1=np.matmul(xtest,w) + mtrain.values
    r20=lossh(yhatbig1,ytest,mu)
    for t in range(max_iter):
        vold=v
        w_perv=w
        w= np.asarray(v-(gamma*f_gradh(v,X,y,mu)))
        mu1=l1*gamma
        w=func(groups,nc,w,mu1)
        w = np.asarray([float(w1) for w1 in w])
        v=w + t/(t+3)*(w-w_perv)
        if ((np.linalg.norm(v-vold)**2) < ((np.linalg.norm(vold)**2)*tol) or np.sum(abs(v-vold))==0):
            break
    return v

In [ ]:
###soft_threshodl

def soft_threshodl(groups,nc,w,mu):
    val =[float(np.sign(w1)*np.max(np.abs(w1)-mu,0)) for w1 in w]
    return np.array(val).reshape((len(val),1))

In [ ]:
####lossh
def lossh(y,yhat,mu):
    r= abs(yhat['RET']-y)
    l= np.zeros(len(r))
    ind = (r>mu)
    l[ind] = 2*mu*r[ind]-mu*mu
    ind = (r<= mu)
    l[ind]  = r[ind]*r[ind]
    return np.mean(l)

In [ ]:
for m in range(len(start_dates)):

    start_date = start_dates[m]

    end_date =  start_date + pd.offsets.DateOffset(years=18)

    ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytrain = Y_all[ind_training]

    xtrain = X_all[ind_training]

    ####for validation

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=12)

    ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytest = Y_all[ind_vali]
    xtest= X_all[ind_vali]

    #####for test  oos

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=1)
    ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
    yoos = Y_all[ind_test]
    xoos= X_all[ind_test]

    #######Monthly Demean %%%
    ytrain_demean= ytrain - np.mean(ytrain);
    ytest_demean=ytest-np.mean(ytest);
    mtrain=np.mean(ytrain);
    mtest=np.mean(ytest);

    sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
    for i in range(len(xtrain.columns)):
        s=np.std(xtrain.iloc[:,[i]])
        colnames = xtrain.columns
        if s.values > 0:
            colname = colnames[i]
            xtrain.loc[:,colname]=xtrain[colname]/s.values
            xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
            xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
            sd[i]=s.values
    XX =np.matmul(np.transpose(xtrain),xtrain)
    U,S,V=svd(XX)
    #### singular value decomposition such that XX=U*S*V'
    L=S[0]
    ###disp 'Lasso L = '
    ####disp(L)
    Y=ytrain_demean;
    XY= np.matmul(np.transpose(xtrain),Y)
    groups=0
    nc=0
            
    #### Start to Train %%%
   
    lamv=np.arange(-2,4,0.1)
    alpha=1
    r=np.zeros((3,len(lamv)))
    y_ret=ytest['RET'].reset_index(drop=True)
    y_ret1=ytrain['RET'].reset_index(drop=True)
    y_ret2=yoos['RET'].reset_index(drop=True)


    for j in range(len(lamv)):
        l2=10**lamv[j]
        lasso = Lasso(alpha=l2)
        yhatres= lasso.fit(xtrain,ytrain_demean)
        yhatbig1= yhatres.predict(xtest)+ mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        #prediction for oos
        r[0,j]=1-sum((yhatbig1-y_ret)**2)/sum((y_ret-mtrain.values)**2) # oos r2
        yhatbig1=yhatres.predict(xtrain) + mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        r[1,j]=1-(yhatbig1-y_ret1).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
        yhatbig1=yhatres.predict(xoos) + mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        r[2,j]=1-(yhatbig1-y_ret2).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
        


    r2_oos=r[2,np.argmax(r[0,:])]
    r2_is=r[1,np.argmax(r[0,:])]
    l2=10**lamv[np.argmax(r[0,:])]
    lasso = Lasso(alpha=l2)
    yhatres= lasso.fit(xtrain,ytrain_demean)
    b=pd.DataFrame(yhatres.coef_)

    print('LASSO R2 : '+np.str0(r2_oos))
    print('LASSO R2_is : '+np.str0(r2_is))
    
    b=proximalH_l(groups,nc,xtest,mtrain,ytest,b,xtrain,ytrain_demean,mu,tol,L,l2,soft_threshodl)
    yhatbig1 = np.matmul(xoos,b)+mtrain.values
    r2_oos=1- np.linalg.norm(yhatbig1-yoos['RET'])**2 / np.linalg.norm((yoos['RET']-mtrain.values))**2
    yhatbig1 = np.matmul(xtrain,b) + mtrain.values
    r2_is = 1 - np.linalg.norm(yhatbig1-ytrain['RET'])**2 / np.linalg.norm((ytrain['RET']-mtrain.values))**2
    b=pd.DataFrame(b)
    print('LASSO+H R2 : '+np.str0(r2_oos) )

<!--
LASSO R2 : 0.5618758011505414
LASSO R2_is : -2.220446049250313e-15
LASSO+H R2 : -13231.55186630835
LASSO R2 : 0.791334380349793
LASSO R2_is : -8.215650382226158e-15
LASSO+H R2 : -113.37696994403363
LASSO R2 : 0.8135787276602633
LASSO R2_is : -1.1546319456101628e-14
LASSO+H R2 : -91.19932662279655
LASSO R2 : 0.6712708722341254
LASSO R2_is : -3.774758283725532e-15
LASSO+H R2 : -15.808291531700895
LASSO R2 : 0.6175672682393455
LASSO R2_is : -1.532107773982716e-14
LASSO+H R2 : -21.61501816034154
LASSO R2 : 0.8836687807482796
LASSO R2_is : -4.440892098500626e-15
LASSO+H R2 : -18.26452482482396
LASSO R2 : 0.8816018788115152
LASSO R2_is : 1.2878587085651816e-14
LASSO+H R2 : -25.858466116550805
LASSO R2 : 0.8764023940613506
LASSO R2_is : 6.994405055138486e-15
LASSO+H R2 : -5.364301724927666
LASSO R2 : 0.8988952383748445
LASSO R2_is : 4.9960036108132044e-15
LASSO+H R2 : -363.6111906558954
LASSO R2 : 0.899925812234136
LASSO R2_is : -1.509903313490213e-14
LASSO+H R2 : -1017.5354692534502
LASSO R2 : 0.865727115062263
LASSO R2_is : -1.9761969838327786e-14
LASSO+H R2 : -850.4572946395298
LASSO R2 : 0.7540483035933313
LASSO R2_is : -7.549516567451064e-15
LASSO+H R2 : -336.8435648490142
LASSO R2 : 0.7127222900468251
LASSO R2_is : 1.2323475573339238e-14
LASSO+H R2 : -33.337338998110056
LASSO R2 : 0.7481107564855115
LASSO R2_is : 2.7755575615628914e-14
LASSO+H R2 : -1.600554753481911
LASSO R2 : 0.8404789635637873
LASSO R2_is : 1.609823385706477e-14
LASSO+H R2 : -0.755792227907494
LASSO R2 : 0.8826133198528734
LASSO R2_is : 1.8762769116165146e-14
LASSO+H R2 : -0.21801419827355994
LASSO R2 : 0.9268850337157866
LASSO R2_is : 1.942890293094024e-14
LASSO+H R2 : -0.30450809678394086
LASSO R2 : 0.9707383241426257
LASSO R2_is : 5.995204332975845e-15
LASSO+H R2 : -0.3218146800467754
LASSO R2 : 0.9668265383494759
LASSO R2_is : 2.0872192862952943e-14
LASSO+H R2 : -0.1213886217808704
LASSO R2 : 0.9669078058373662
LASSO R2_is : -2.4424906541753444e-14
LASSO+H R2 : -0.05936537507378947
LASSO R2 : 0.9677117757903464
LASSO R2_is : 4.1300296516055823e-14
LASSO+H R2 : -0.000747564974955317
LASSO R2 : 0.9356136435388934
LASSO R2_is : -3.6193270602780103e-14
LASSO+H R2 : 0.0630407116565862
LASSO R2 : 0.9255911956162547
LASSO R2_is : 2.7533531010703882e-14
LASSO+H R2 : -0.07201033541048174
LASSO R2 : 0.9773278842816471
LASSO R2_is : -5.5067062021407764e-14
LASSO+H R2 : -0.022548647087312634
LASSO R2 : 0.9681875663118822
LASSO R2_is : -5.773159728050814e-15
LASSO+H R2 : 0.005209634458331003
LASSO R2 : 0.9807186885727028
LASSO R2_is : 1.4099832412739488e-14
LASSO+H R2 : -0.048136228116649526
LASSO R2 : 0.9823491892298412
LASSO R2_is : 4.107825191113079e-15
LASSO+H R2 : 0.0004107272280275964
LASSO R2 : 0.9861978426740109
LASSO R2_is : -2.220446049250313e-14
LASSO+H R2 : -0.019839744688194605
LASSO R2 : 0.982101122170377
LASSO R2_is : -3.774758283725532e-15
LASSO+H R2 : 0.02037271028373233
LASSO R2 : 0.9813564358962984
LASSO R2_is : 1.1657341758564144e-14
LASSO+H R2 : -0.02952216878173264